# Implementação MLP para dataset

In [1]:
# Função de ativação Logit

def f_net(activation):
     return 1.0 / (1.0 + np.exp(-activation))

In [2]:
# Derivada da Logit

def f_prime_net(f_net):
    return f_net*(1-f_net)

In [3]:
# Etapa Foward

def f_Forward(Xp,w_hidden_layer1,w_hidden_layer2,w_output_layer,net_hidden1,fnet_hidden1,net_hidden2,fnet_hidden2,net_out, fnet_out):
    # Hidden Layer 1
    net_hidden1  = np.sum(w_hidden_layer1*Xp,axis=1)
    fnet_hidden1 = f_net(net_hidden1)
    f_prime_hp1  = f_prime_net(fnet_hidden1)
    
    # Hidden Layer 2
    net_hidden2  = np.sum(np.r_[fnet_hidden1, 1]*w_hidden_layer2,axis=1)
    fnet_hidden2 = f_net(net_hidden2)
    f_prime_hp2 = f_prime_net(fnet_hidden2)
    
    # Output Layer
    net_out = np.sum(np.r_[fnet_hidden2, 1]*w_output_layer,axis=1)
    fnet_out =  f_net(net_out)
    f_prime_op = f_prime_net(fnet_out)
    
    return net_hidden1, fnet_hidden1, net_hidden2, fnet_hidden2, net_out, fnet_out, f_prime_op, f_prime_hp1, f_prime_hp2



In [4]:
# Backpropagation

def mlp_backpropagation(X,Y,n_inputs,n_hidden1,n_hidden2,n_outputs ,eta ,Epoch,alpha):
    # Vetor de entrada com acréscimo de coluna para bias
    Xp = np.concatenate((X, np.reshape(np.ones(len(X)),(len(X),1))), axis=1)

    # Inicialização dos pesos de forma aleatória
    var_w1 = w_hidden_layer1_ant = w_hidden_layer1 = np.array([[random.uniform(-0.5,0.5) for i in range(n_inputs+1)] for i in range(n_hidden1)])
    var_w2 = w_hidden_layer2_ant = w_hidden_layer2 = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden1+1)] for i in range(n_hidden2)])
    var_wo = w_output_layer_ant = w_output_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden2+1)]     for i in range(n_outputs)])

    ## Hidden Layer 1
    net_hidden1  = np.zeros(w_hidden_layer1.shape)
    fnet_hidden1 = np.zeros(w_hidden_layer1.shape)

    ## Hidden Layer 2
    net_hidden2  = np.zeros(w_hidden_layer2.shape)
    fnet_hidden2 = np.zeros(w_hidden_layer2.shape)
    
    ## Output Layer
    net_out  = np.zeros(n_outputs)
    fnet_out = np.zeros(n_outputs)
    # Vetor de saída da hidden com acréscimo de coluna para bias

    counter = 0

    while counter <= Epoch :
        squaredError = 0
        for i in range(len(X)):
            net_hidden1, fnet_hidden1, net_hidden2, fnet_hidden2, net_out, fnet_out, f_prime_op, f_prime_hp1, f_prime_hp2 = f_Forward(Xp[i],w_hidden_layer1,w_hidden_layer2,w_output_layer,net_hidden1,fnet_hidden1,net_hidden2,fnet_hidden2,net_out, fnet_out)  
        
        
            error = Y[i] - fnet_out
            squaredError = squaredError + np.sum(error**2)

            #Cálculo dos deltas
            # Output
            delta_op  = error * f_prime_op   
            # Hidden 2
            w_o_kj_2   = w_output_layer[:,0:w_output_layer.shape[1]-1]
            delta_h_p2 = f_prime_hp2*np.dot(w_o_kj_2.T,delta_op)
            # Hidden 1
            w_o_kj_1   = w_hidden_layer2[:,0:w_hidden_layer2.shape[1]-1]
            delta_h_p1 = f_prime_hp1*np.dot(w_o_kj_1.T,delta_h_p2)
            
            #Treinamento
            w_output_layer  = w_output_layer  + eta * np.dot(np.reshape(delta_op,(len(w_output_layer),1)), np.reshape(np.r_[fnet_hidden2,1],(1,n_hidden2+1))) + alpha * var_wo 
            w_hidden_layer2 = w_hidden_layer2 + eta * np.dot(np.reshape(delta_h_p2,(n_hidden2,1)),np.reshape(np.r_[fnet_hidden1,1],(1,n_hidden1+1))) + alpha * var_w2  
            w_hidden_layer1 = w_hidden_layer1 + eta * np.dot(np.reshape(delta_h_p1,(n_hidden1,1)),np.reshape(Xp[i],(1,len(X[0])+1))) + alpha * var_w1
            
            # Termo momentum
            var_wo =  w_output_layer - w_output_layer_ant 
            w_output_layer_ant = w_output_layer.copy()
            
            var_w2 = w_hidden_layer2 - w_hidden_layer2_ant 
            w_hidden_layer2_ant = w_hidden_layer2.copy()
            
            var_w1 = w_hidden_layer1 - w_hidden_layer1_ant 
            w_hidden_layer1_ant = w_hidden_layer1.copy()
                     
           
            
        squaredError = squaredError / len(X)
        print(squaredError)
        counter = counter + 1
    
    return  squaredError , w_output_layer, w_hidden_layer1, w_hidden_layer2, fnet_out, counter

In [5]:
# Predição de novos dados

def f_predict(Xp,w_hidden_layer1,w_hidden_layer2,w_output_layer):
   
    return np.round(f_net(np.sum(np.r_[f_net(np.sum(np.r_[f_net(np.sum(w_hidden_layer1*Xp,axis=1)), 1]*w_hidden_layer2,axis=1)), 1]*w_output_layer,axis=1)))


In [8]:
import numpy as np
from math import exp
import random
import pandas as pd
#tratamentos de dataframes
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

random.seed(123)

# Importação do conjunto de dados Wine sem cabeçalho
df = pd.read_csv("/home/jovyan/MLPxRBF/wine.csv",delimiter=',',header=None)

# Nomeação das colunas
dict_colnames = {
0: 'type',
1: 'alcohol',
2: 'malic_acid',
3: 'ash',
4: 'alcalinity_ash',
5: 'magnesium',
6: 'total_phenols',
7: 'flavanoids',
8: 'nonflavanoid_phenols',
9: 'proanthocyanins',
10: 'color_intensity',
11: 'hue',
12: 'OD280_OD315_diluted_wines',
13: 'proline'
   }
    
df = df.rename(columns= dict_colnames)

# Criação do vetor de base canônica
df['p1'] = np.where(df['type']==1, 1, 0)
df['p2'] = np.where(df['type']==2, 1, 0)
df['p3'] = np.where(df['type']==3, 1, 0)

# Separação entre dados de treino e teste
lista = ['type','p1','p2','p3']
lista2 = ['p1','p2','p3']

x_train, x_test, y_train, y_test = train_test_split(df.drop(lista, axis = 1), 
                                                    df[lista2], 
                                                    test_size = 0.3, 
                                                    random_state = random.seed(123))

# Normalização dos dados
x_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)


# Matriz para treinar a rede
X = x_train_norm.copy()
Y = np.array(y_train.copy())

## Dimensões da Rede
n_inputs = len(X[0])
n_outputs = len(Y[0])
n_hidden1 = 3
n_hidden2 = 3


Erro, w_output_layer, w_hidden_layer1, w_hidden_layer2, fnet_out, counter = \
    mlp_backpropagation(X,Y,n_inputs,n_hidden1,n_hidden2,n_outputs,eta = 0.01,Epoch=1000,alpha=0.3)


0.6801272868393718
0.6755078297081747
0.6719034211787319
0.6691824149063555
0.6671295726966383
0.66557763226358
0.6643981575748077
0.6634938078102776
0.6627917000541728
0.6622378248180769
0.661792487321041
0.6614266937070211
0.661119357101444
0.6608551806868407
0.6606230789421754
0.6604150144912279
0.6602251490049695
0.660049227566843
0.659884134476577
0.6597275738047246
0.6595778401212418
0.6594336540960176
0.6592940446186204
0.659158264210294
0.6590257282387703
0.6588959711488158
0.6587686148635041
0.6586433459008467
0.6585198987418533
0.6583980436921785
0.6582775779818856
0.6581583192052761
0.6580401004571117
0.6579227667027866
0.6578061720493528
0.6576901776766985
0.6575746502544625
0.6574594607177787
0.6573444833092436
0.6572295948192126
0.6571146739744961
0.6569996009385322
0.6568842568956597
0.6567685236990625
0.6566522835670804
0.6565354188163847
0.6564178116232858
0.6562993438065522
0.6561798966266669
0.6560593505976092
0.6559375853081499
0.6558144792502923
0.6556899096530188


0.055522059677161645
0.05506994386180332
0.05462373591992805
0.0541833364572804
0.053748647852785686
0.053319574231345784
0.052896021436839705
0.05247789700533481
0.05206511013851623
0.051657571677340995
0.051255194075924486
0.05085789137566264
0.050465579179597625
0.05007817462703063
0.04969559636838728
0.049317764540339364
0.048944600741187186
0.04857602800650662
0.048211970785063445
0.0478523549149989
0.04749710760028817
0.04714615738747449
0.0467994341426823
0.046456869028909505
0.046118394483600826
0.045783944196504965
0.04545345308781456
0.04512685728659108
0.04480409410947483
0.04448510203968024
0.044169820706276855
0.04385819086375514
0.043550154371877474
0.04324565417581322
0.04294463428655743
0.042647039761632
0.04235281668606784
0.042061912153667086
0.0417742742485439
0.04148985202694131
0.04120859549932351
0.0409304556127407
0.040655384233464514
0.040383334129891896
0.040114258955714834
0.039848113233353966
0.03958485233765322
0.03932443247983228
0.03906681069169513
0.03881

0.009626048198573456
0.009603085791619257
0.009580224060774155
0.009557462374850481
0.009534800107733022
0.009512236638330226
0.009489771350525936
0.009467403633131699
0.00944513287983962
0.00942295848917562
0.009400879864453345
0.00937889641372837
0.009357007549753119
0.009335212689932164
0.009313511256277988
0.009291902675367125
0.009270386378297156
0.009248961800643813
0.009227628382418572
0.00920638556802677
0.009185232806226306
0.00916416955008649
0.009143195256947604
0.00912230938838083
0.009101511410148415
0.009080800792164558
0.009060177008456523
0.009039639537126203
0.009019187860312097
0.008998821464151732
0.008978539838744417
0.008958342478114565
0.008938228880175085
0.008918198546691384
0.008898250983245924
0.00887838569920264
0.00885860220767222
0.008838900025477636
0.008819278673119818
0.008799737674743914
0.008780276558105858
0.008760894854539349
0.008741592098922928
0.008722367829647694
0.008703221588585193
0.008684152921055796
0.008665161375797167
0.008646246504933251


## Predito

In [65]:
Xp = np.concatenate((x_test_norm, np.reshape(np.ones(len(x_test_norm)),(len(x_test_norm),1))), axis=1)

In [67]:
Y_pred =  np.zeros(y_test.shape)
for i in range(len(Xp[:,])):
    Y_pred[i] = f_predict(Xp[i],w_hidden_layer1,w_hidden_layer2,w_output_layer)

In [68]:
np.argmax(Y_pred,axis=1)

array([1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1,
       2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2,
       0, 2, 2, 2, 0, 1, 0, 0, 1, 2])

## Real

In [69]:
np.argmax(np.array(y_test),axis=1)

array([1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1,
       2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2,
       0, 2, 2, 2, 0, 1, 0, 0, 1, 2])

# MLP

## Matriz de Confusão

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

array([[14,  0,  0],
       [ 1, 23,  0],
       [ 0,  0, 16]])

## Acurácia

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

0.9814814814814815